In [6]:
import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import requests
import PyPDF2 as pdf
import io
import os
import re
from gsheet import create
from gsheet import update
import weblist
import time

## Step1: Scrape the RSS Feed

In [2]:
# URL of the New Orleans RSS feed to scrape
rss_url = "https://www.mvn.usace.army.mil/DesktopModules/ArticleCS/RSS.ashx?ContentType=4&Site=417&isdashboardselected=0&max=3000"

# Parse the RSS feed using feedparser
rss_parsed = feedparser.parse(rss_url)

# Create a df to save everything into
rss_df = pd.DataFrame(rss_parsed.entries)

# Clean the df for New Orleans
rss_df = rss_df[["title", "link", "summary", "published"]].rename(
    columns = {"title": "web_title", "link": "web_link", "summary": "web_summary", "published": "published_date"}, 
    copy = False)

## Step2: Scrape the webpage

### Define webpage scraping functions

In [3]:
def get_web_expire_date(soup):
    if soup.find("div", "expire") is None:
        web_expire_date = None
    else:
        try:
            web_expire_date = re.search(r'(?<=:\s).+', soup.find_all("div", "expire")[0].get_text()).group()
        except Exception as e:
            web_expire_date  = "ERROR: " + str(e)
    return web_expire_date


def get_web_pdf_url(soup):
    try:
        pdf_end = soup.findAll('a', {"class": "link"})[2]['href']
        pdf_url = "https://www.mvn.usace.army.mil" + pdf_end
    except Exception as e:
        pdf_url = "ERROR: " + str(e)
    finally:
        return pdf_url

    
def get_web_special(soup):
    try:
        notice_type = soup.find_all("a", {"class":"link"})[2].get_text()
        if "Special" in notice_type:
            special_public_notice = "Yes"   
        else:
            special_public_notice = "No"
    except Exception as e:
        special_public_notice = "ERROR: " + str(e)
    finally:
        return special_public_notice

    
def get_web_text(soup):
    try:
        body = soup.find_all("div", {"itemprop": "articleBody"})[0]
        if body.find("p") is None:
            web_text = body.get_text()
        else:
            web_text = body.get_text().replace(u'\xa0', u'').replace("\n", "")
    except:
        web_text = "Error"
    finally:
        return web_text

    
def get_web_applicant(web_text):
    # Get all info in "NAME OF APPLICANT"
    if any(w in web_text for w in ["APPLICANT", "Applicant"]):
        try:
            web_applicant_contents = re.search(r'(?<=(APPLICANT|Applicant):)\s*.+(?=(LOCATION|Location))', web_text).group().strip()
            # Extract applicant and contractor
            if web_applicant_contents.find("c/o") != -1:
                try:
                    web_applicant = re.search(r'.+?(?=\,* c/o)', web_applicant_contents).group().strip()
                except Exception as e:
                    web_applicant = "ERROR: " + str(e)
                try:    
                    web_contractor = re.search(r'(?<=c/o( |:)).+?(?=(,?\s?Post|,?\s?PO|,?\s?P\.O\.|,?\s*\d|,?\s?[Aa][tT]))', web_applicant_contents).group().strip()
                except Exception as e:
                    web_contractor = "ERROR: " + str(e)
            else:
                web_contractor = None
                try:
                    web_applicant = re.search(r'.+?(?=(,?\s?Post|,?\s?PO|,?\s?P\.O\.|,?\s?\d|,?\s?[Aa][tT]))', web_applicant_contents).group().strip()
                except Exception as e:
                    web_applicant = "ERROR: " + str(e)   
        except Exception as e:
            web_applicant_contents = web_applicant = web_contractor = "Error: " + str(e)
    else:
        web_applicant_contents = web_applicant = web_contractor = None
    return web_applicant_contents, web_applicant, web_contractor

def get_web_location(web_text):
    if any(w in web_text for w in ["LOCATION", "Location"]):
        try:
            web_location = re.search(r'(?<=(LOCATION OF WORK|Location of Work):)\s*.+(?=(CHARACTER OF WORK|Character of Work))', web_text).group().strip()
        except Exception as e:
            web_location = "ERROR: " + str(e)
    else:
        web_location = None
    return web_location
        
    
def get_web_character_mitigation(web_text):
    if web_text.find(r'(Mitigation|MITIGATION)') != -1:
        if any(w in web_text for w in ["CHARACTER", "Character"]):
            try:
                web__character = re.search(r'(?<=(CHARACTER OF WORK|Character of Work):)\s*.+(?=(MITIGATION|Mitigation))', web_text).group().strip()
            except Exception as e:
                web__character = "ERROR: " + str(e)
        else:
            web__character = None
        try:
            web_mitigation = re.search(r'(?<=(MITIGATION|Mitigation):)\s*.+', web_text).group().strip()
        except Exception as e:
            web_mitigation = "ERROR: " + str(e)
    else:
        web_mitigation = None                                     
        try:
            web_character = re.search(r'(?<=(CHARACTER OF WORK|Character of Work):)\s*.+', web_text).group().strip()
        except Exception as e:
            web_character = "ERROR: " + str(e)
    return web_character, web_mitigation        


def web_extraction(web_url):
    
    req = requests.get(web_url)
    content = req.text
    soup = BeautifulSoup(content, 'html.parser')

    # Get the pdf links
    pdf_url = get_web_pdf_url(soup)

    # Check if the public notice is "Special Public Notice" or "Joint Public Notice"
    web_special = get_web_special(soup)

    # Get expiration date
    web_expire_date = get_web_expire_date(soup)

    # Extract webpage body
    web_text = get_web_text(soup)

    if web_text != "Error":

        # Get applicant and contractor
        web_applicant = get_web_applicant(web_text)

        # Get location
        web_location = get_web_location(web_text)

        # Get character of work and mitigation if any
        web_character_mitigation = get_web_character_mitigation(web_text)
        
    else:
        # Assign "Error" to all fields inside of website body.
        web_applicant = "ERROR", "ERROR", "ERROR"
        web_location = "ERROR"
        web_character_mitigation = "ERROR", "ERROR"
        
    return [pdf_url, web_special, web_expire_date, web_applicant[0], web_applicant[1], web_applicant[2], \
 web_location, web_character_mitigation[0], web_character_mitigation[1]]

### Get the extracted results

In [4]:
web_df = pd.DataFrame([web_extraction(x) for x in rss_df["web_link"]],
                            columns = ['pdf_url',
                                       'web_special',
                                       'web_expire_date',  
                                       'web_applicant_contents', 
                                       'web_applicant', 
                                       'web_contractor', 
                                       'web_location', 
                                       'web_character', 
                                       'web_mitigation'])
# Merge with RSS feed table
rss_web_df = rss_df.join(web_df)

# Add a new column to track the number of errors for each webpage
#rss_web_df["error"] = rss_web_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

rss_web_df

,web_title,web_link,web_summary,published_date,pdf_url,web_special,web_expire_date,web_applicant_contents,web_applicant,web_contractor,web_location,web_character,web_mitigation
0,MVN-2015-01960-CM,https://www.mvn.usace.army.mil/Missions/Regula...,PERMIT MODIFICATION FOR THE CONSTRUCTED ASCENS...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/25/2023,"Ascension Pipeline LLC, c/o SWCA Environmental...",Ascension Pipeline LLC,SWCA Environmental Consultants,Beginning at the Marathon Refinery in Garyvill...,A permit for the work described herein was pre...,None
1,MVN-2014-02629-CM,https://www.mvn.usace.army.mil/Missions/Regula...,MISSISSIPPI RIVER DREDGING WITH THE CONSTRUCTI...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,10/3/2023,"The Mosaic Company, c/o Matrix PDM, Attn: Bill...",The Mosaic Company,M,"In the Mississippi River, LDB, River Mile 160....",The applicant has requested Department of the ...,None
2,MVN-2019-01224-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED SHORELINE PROTECTION IN TERREBONNE PA...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/25/2023,"Terrebonne Parish Consolidated Government, c/o...",Terrebonne Parish Consolidated Government,"T. Baker Smith, LLC","Along Lake Chien and Lake Tambour, in Terrebon...",Construct approximately 3.54 linear miles of o...,None
3,MVN-2007-00574-EG,https://www.mvn.usace.army.mil/Missions/Regula...,BARGE FLEET EXPANSION ON THE MISSISSIPPI RIVER...,"Mon, 28 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/27/2023,"Cooper Consolidated, c/o Gulf South Engineerin...",Cooper Consolidated,Gulf South Engineering & Testing,"At approximately Latitude: 30.046667, Longitud...",Expand an existing barge fleet to add one (1) ...,None
4,MVN-2022-00799-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED RV PARK ALONG PORT ALLEN LOCK<br />,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"Scott Thomas Development, LLC c/o: Trusted Com...","Scott Thomas Development, LLC","Trusted Compliance, LLC","Located in Brusly, Louisiana, within West Bato...",The applicant has requested Department of the ...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MVN 2018-00250-WLL,https://www.mvn.usace.army.mil/Missions/Regula...,COMMONWEALTH LIQUID NATURAL GAS (LNG) FACILITY...,"Mon, 11 May 2020 11:33:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Commonwealth LNG, LLC, c/o TRC Environmental C...","Commonwealth LNG, LLC",TRC Environmental Corpor,"In Cameron Parish, Louisiana; LNG Facility loc...",The applicant has requested a Department of th...,None
496,MVN 2019-01282-CE,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED BURNSIDE COMMERCIAL DEVELOPMENT ON SO...,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Tower Capital, c/o Hydrik Wetlands Consultants...",Tower Capital,"Hydrik Wetlands Consultants, LLC.",In wetlands and an unnamed canal (Latitude: 30...,The applicant has requested authorization from...,None
497,MVN-2016-0269-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TRANSMISSION LINE IN ST. TAMMANY PARISH<br />,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Washington St. Tammany Electric Cooperative, c...",Washington St. Tammany Electric Cooperative,"Providence Engineering and Environmental, LLC",Project site is located approximately 10.27 mi...,Construct an approximate nine-mile-long transm...,None
498,MVN-2019-0518-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,INSTALL AND MAINTAIN 4-12” PIPELINE IN ST. CHA...,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"International-Matex Tank Terminals, c/o Rambol...",International-Matex Tank Terminals,Ramboll,Project site is located from 

## Step3: Scrap PDFs

### Define PDF scraping functions

In [34]:
district_dic = {"MVN": "New Orleans District",
               "SWG": "Galveston District",
               "SAM": "Mobile District",
               "SAJ": "Jacksonville District"}

# Seperate pdf texts into big chuncks:

## Permit application # + district code + district Name
def get_pdf_app_num_dist(pdf_text):
    try:
        permit_application_number = re.search(r'(?<=Application).*(?=PUBLIC)', pdf_text).group().replace("#", "").replace(":", "").strip()
        district_code = permit_application_number[0:3]
        district_name = district_dic[district_code]
    except Exception as e:
        permit_application_number = "ERROR: " + str(e)
        district_code = "ERROR: cannot get permit application number"
        district_name = "ERROR: cannot get permit application number"
    finally:
        return permit_application_number, district_code, district_name

## Manager name + phone + email
def get_pdf_manager(pdf_text):
    try:
        manager_name = re.search(r'(?<=Project Manager)[\:a-zA-Z\s\.]*', pdf_text).group().replace(":", "").strip()
    except Exception as e:
        manager_name = "ERROR: " + str(e)
    try:
        manager_phone = re.search(r'\(\d{3}\)\s{1,3}\d{3}\s?-?\s?\d{4}', pdf_text).group().strip()
    except Exception as e:
        manager_phone = "ERROR: " + str(e)
    try:
        manager_email = re.search(r'[\w\.-]+@us\s?a\s?c\s?e\s?\.army\.m\s?i\s?l', pdf_text).group().strip()
    except Exception as e:
        manager_email = "ERROR: " + str(e)
    return manager_name, manager_phone, manager_email
    
## Location of work
def get_pdf_location(pdf_text):
    if pdf_text.find("LOCATION OF WORK") != -1:
        try:
            pdf_location = re.search(r'((?<=LOCATION OF WORK).*(?=CHARACTER OF WORK))', pdf_text).group().replace(":", "").strip()
        except Exception as e:
            pdf_location = "ERROR: " + str(e)
    else:
        pdf_location = None
    return pdf_location

## Character of work 
def get_pdf_character(pdf_text):
    if pdf_text.find("CHARACTER OF WORK") != -1:
        try:
            pdf_character = re.search(r'((?<=CHARACTER OF WORK).*(?=(MITIGATION|The comment period)))', pdf_text).group().replace(":", "").strip()
        except Exception as e:
            pdf_character = "ERROR: " + str(e)
    else:
        pdf_character = None
    return pdf_character

# Extract fields from paragraphs

## From location of work
def get_pdf_county_parish(pdf_text):
    if pdf_text.find("County") != -1:
        try:
            county = re.search(r'((?<=, ).*County)', pdf_text).group().strip()
        except Exception as e:
            county = "ERROR: " + str(e)
    else:
        county = None
    if pdf_text.find("Parish") != -1:
        try:
            parish = re.search(r'((?<=in ).{1,100}(?= Parish))', pdf_text).group().strip()
        except Exception as e:
            parish = "ERROR: " + str(e)
    else:
        parish = None
    return county, parish


def get_pdf_hydrologic(pdf_text):
    if pdf_text.find(r'Hydrologic Unit Code') != -1:
        try:
            hydrologic_unit_code = re.search(r'(?<=Hydrologic Unit Code(:|\s))[\s|\d]*', pdf_text).group().strip()
        except Exception as e:
            hydrologic_unit_code = "ERROR: " + str(e)
    else:
        hydrologic_unit_code = None
    return hydrologic_unit_code

## From character of work
def get_pdf_acreage(pdf_text):
    if all(w in pdf_text for w in ["acre", "impact"]):
        try:
            acreage = re.findall(r'(\d*\.\d*-?\s?(?=acres of))', pdf_text)
            acreage = [a.strip() for a in acreage]
            # wetland_type = re.search(r'(?<=acres of).+? wetlands', pdf_text).group().strip()
        except Exception as e:
            acreage = "ERROR: " + str(e)
            # wetland_type = "ERROR: " + str(e)
    else:
        acreage = None
        # wetland_type = None
    return acreage

def get_lon_lat(pdf_text):
    if any(w in pdf_text for w in ["Longitude", "long"]):
        try:
            lon = re.findall(r'(?<=[-W])\s*\d{2}\.\d+', pdf_text)
        except Exception as e:
            lon = "ERROR: " + str(e)
    else:
        lon = None
    if any(w in pdf_text for w in ["Latitude", "lat"]):
        try:
            lat = re.findall(r"(?<=[^-W][^-\d])\d{2}\.\d+", pdf_text)
        except Exception as e:
            lat = "ERROR: " + str(e)
    else:
        lat = None
    return lon, lat


def get_wqc(pdf_text):
    if pdf_text.find("WQC") != -1:
        try:
            wqc = re.search(r'(?<=WQC)[\d\s\:]*-[\s\d]*', pdf_text).group().strip().replace(" ", "")
        except Exception as e:
            wqc = "ERROR: " + str(e)
    else:
        wqc = None
    return wqc


def get_coastal_use_permit(pdf_text):
    if pdf_text.find("Natural Resource’s Coastal Resources Program") != -1:
        try:
            coastal_use_permit_list = re.findall(r'P\d{8}', pdf_text)
            coastal_use_permit = ", ".join(coastal_use_permit_list)
        except Exception as e:
            coastal_use_permit = "ERROR: " + str(e)
    else:
        coastal_use_permit = None
    return coastal_use_permit


def pdf_read(pdf_url):
    try:
        # Download the PDF content as a bytes object
        pdf_bytes = requests.get(pdf_url).content

        # Create a PyPDF2 PdfFileReader object from the PDF content
        pdf_reader = pdf.PdfReader(io.BytesIO(pdf_bytes))

        # Extract text from all pages except appendix in the PDF file
        pdf_full = []
        for p in range(len(pdf_reader.pages)):
            pdf_p = pdf_reader.pages[p].extract_text()
            pdf_full.append(pdf_p)
            if "Enclosure" in pdf_p:
                break

        pdf_text = "".join(pdf_full).replace("\n", "")
        
    except Exception as e:
        pdf_text = "ERROR: " + str(e)
    finally:
        return pdf_text

# Read in PDF as texts and extract fields
def pdf_extraction(pdf_url):
    
    pdf_text = pdf_read(pdf_url)
    
    # standardized public notice
    if  pdf_text.find("ERROR") == -1:

        pdf_app_num_dist = get_pdf_app_num_dist(pdf_text)

        pdf_manager = get_pdf_manager(pdf_text)

        pdf_location = get_pdf_location(pdf_text)

        if pdf_location is None:
            county = parish = hydrologic_unit_code = lon = lat = None
        elif "ERROR" in pdf_location:
            county = parish = hydrologic_unit_code = lon = lat = "ERROR: cannot extract location of work"
        else:
            county = get_pdf_county_parish(pdf_location)[0]
            parish = get_pdf_county_parish(pdf_location)[1]
            hydrologic_unit_code = get_pdf_hydrologic(pdf_location)
            lon = get_lon_lat(pdf_location)[0]
            lat = get_lon_lat(pdf_location)[1]

        pdf_character = get_pdf_character(pdf_text)

        if pdf_character is None:
            acreage = None
        elif "ERROR" in pdf_character:
            acreage = "ERROR: cannot extract character of work"
        else:
            acreage = get_pdf_acreage(pdf_character)
        
        wqc = get_wqc(pdf_text)
        
        cup = get_coastal_use_permit(pdf_text)
    
    # Special public notice
    else:
        pdf_app_num_dist = ["ERROR: fail to read pdf " + pdf_text]*3
        pdf_manager = ["ERROR: fail to read pdf " + pdf_text]*3
        pdf_location = pdf_character = county = parish = hydrologic_unit_code = lon = lat = acreage = wqc = cup = "ERROR: fail to read pdf " + pdf_text
        
    return [pdf_app_num_dist[0], pdf_app_num_dist[1], pdf_app_num_dist[2], pdf_manager[0], pdf_manager[1], \
            pdf_manager[2], pdf_location, pdf_character, county, parish, hydrologic_unit_code, lon, lat, acreage, wqc, cup]


In [35]:
# test

pdf_url = "https://www.mvn.usace.army.mil/Portals/56/docs/regulatory/publicnotices/2020-00168-WRR%20PNALL.pdf?ver=TKjkvnv5Zg4fAplngIrBTQ%3d%3d"
start = time.time()
pdf_bytes = requests.get(pdf_url).content
pdf_reader = pdf.PdfReader(io.BytesIO(pdf_bytes))

pdf_full = []
for p in range(len(pdf_reader.pages)):
    pdf_p = pdf_reader.pages[p].extract_text()
    pdf_full.append(pdf_p)
    if "Enclosure" in pdf_p:
        break

pdf_text = "".join(pdf_full).replace("\n", "")
end = time.time()
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

The time of execution of above program is : 444.78440284729004 ms


### Extract PDF fields

In [39]:
pdf_df = pd.DataFrame([pdf_extraction(x) for x in rss_web_df["pdf_url"]],
                            columns = ['pdf_permit_app_num',
                                       'pdf_dist_code',
                                       'pdf_dist_name',  
                                       'pdf_manager_name', 
                                       'pdf_manager_phone', 
                                       'pdf_manager_email',
                                       'pdf_location', 
                                       'pdf_character', 
                                       'pdf_county',
                                       'pdf_parish',
                                       'pdf_hydrologic_unit_code', 
                                       'pdf_longitude',
                                       'pdf_latitude',
                                       'pdf_acreage',
                                       'pdf_wqc',
                                       'pdf_cup'])

# # Merge with RSS feed table
final_df = rss_web_df.join(pdf_df)

# Add a new column to track the number of errors for each webpage
final_df["error"] = final_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

final_df

unknown widths : 
[0, IndirectObject(315, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(318, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(321, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(324, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(327, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(330, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(333, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(336, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(339, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(342, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(345, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(348, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(351, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(354, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(357, 0, 2558174614352)]
unknown widths : 
[0, IndirectObject(360, 0, 2558174614352)]
unknown widths : 
[0, In

,web_title,web_link,web_summary,published_date,pdf_url,web_special,web_expire_date,web_applicant_contents,web_applicant,web_contractor,...,pdf_character,pdf_county,pdf_parish,pdf_hydrologic_unit_code,pdf_longitude,pdf_latitude,pdf_acreage,pdf_wqc,pdf_cup,error
0,MVN-2015-01960-CM,https://www.mvn.usace.army.mil/Missions/Regula...,PERMIT MODIFICATION FOR THE CONSTRUCTED ASCENS...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/25/2023,"Ascension Pipeline LLC, c/o SWCA Environmental...",Ascension Pipeline LLC,SWCA Environmental Consultants,...,A permit for the work descr ibed herein was pr...,None,"Geismar, Louisiana. The project will span Asce...",None,None,None,"[7.169, 5.755]",None,None,3
1,MVN-2014-02629-CM,https://www.mvn.usace.army.mil/Missions/Regula...,MISSISSIPPI RIVER DREDGING WITH THE CONSTRUCTI...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,10/3/2023,"The Mosaic Company, c/o Matrix PDM, Attn: Bill...",The Mosaic Company,M,...,The applicant has requested Department of the ...,None,St. James,None,None,None,[1.5],None,None,3
2,MVN-2019-01224-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED SHORELINE PROTECTION IN TERREBONNE PA...,"Mon, 04 Sep 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/25/2023,"Terrebonne Parish Consolidated Government, c/o...",Terrebonne Parish Consolidated Government,"T. Baker Smith, LLC",...,Construct approximately 3.54 linear miles of o...,None,Terre bonne,080903 02,[90.44004],[29.3488],[6.6],None,None,0
3,MVN-2007-00574-EG,https://www.mvn.usace.army.mil/Missions/Regula...,BARGE FLEET EXPANSION ON THE MISSISSIPPI RIVER...,"Mon, 28 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/27/2023,"Cooper Consolidated, c/o Gulf South Engineerin...",Cooper Consolidated,Gulf South Engineering & Testing,...,Expand an existing barge fleet to add one (1) ...,None,"LaPlace, Louisiana, in St. John the Baptist",08070100,[90.474722],[],None,None,None,0
4,MVN-2022-00799-CF,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED RV PARK ALONG PORT ALLEN LOCK<br />,"Mon, 21 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,9/11/2023,"Scott Thomas Development, LLC c/o: Trusted Com...","Scott Thomas Development, LLC","Trusted Compliance, LLC",...,The applicant has requested Department of the ...,None,"Brusly, Louisiana, within West Baton Rouge",08070300,[91.317175],[30.393175],[7.43],None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MVN 2018-00250-WLL,https://www.mvn.usace.army.mil/Missions/Regula...,COMMONWEALTH LIQUID NATURAL GAS (LNG) FACILITY...,"Mon, 11 May 2020 11:33:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Commonwealth LNG, LLC, c/o TRC Environmental C...","Commonwealth LNG, LLC",TRC Environmental Corpor,...,The applicant has requested a Depar tment of t...,None,ERROR: 'NoneType' object has no attribute 'group',08080206,[93.3527],None,"[118.8, 89.5, 65.7, 9.5, 14.3, 1.5, 1.2, 37.7,...",-200227,None,4
496,MVN 2019-01282-CE,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED BURNSIDE COMMERCIAL DEVELOPMENT ON SO...,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Tower Capital, c/o Hydrik Wetlands Consultants...",Tower Capital,"Hydrik Wetlands Consultants, LLC.",...,The applicant has requested authorization from...,None,"Gonzales , Louisiana, in Ascension",None,[90.916170],[30.203400],"[12.6, 2.8, 0.702, 0.49]",-200218,None,3
497,MVN-2016-0269-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TRANSMISSION LINE IN ST. TAMMANY PARISH<br />,"Mon, 11 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,No,5/30/2020,"Washington St. Tammany Electric Cooperative, c...",Washington St. Tammany Electric Cooperative,"Providence Engineering and Environmental, LLC",...,None,ERROR: cannot extrac

In [40]:
final_df.to_csv("new_orleans.csv")

## Report the results to Google Sheet

In [129]:
# Make the first row in the dataset to have same name as column (in order for the google sheets to have appropriate column names)
new_header = pd.DataFrame({'web_title': ['web_title'],
                           'web_link': ['web_link'],
                           'web_summary': ['web_summary'],
                           'published_date': ['published_date'],
                           'pdf_url': ['pdf_url'],
                           'web_special': ['web_special'],
                           'web_expire_date': ['web_expire_dat'],
                           'web_applicant_contents': ['web_applicant_contents'],
                           'web_applicant': ['web_applicant'],
                           'web_contractor': ['web_contractor'],
                           'web_location': ['web_location'],
                           'web_character': ['web_character'],
                           'web_mitigation': ['web_mitigation'], 
                           'permit application number': ['permit application number'],
                           'district code': ['district code'],
                           'district name': ['district name'],
                           'manager name': ['manager name'],
                           'manager phone': ['manager phone'],
                           'manager email': ['manager email'],
                           'pdf_location': ['pdf_location'],
                           'pdf_character': ['pdf_character'],
                           'county': ['county'],
                           'parish': ['parish'],
                           'hydrologic_unit_code': ['hydrologic_unit_code'],
                           'acreage': ['agreage'],
                           'error': ['error']})

# Concatenate the new row with the existing DataFrame
final_gs = pd.concat([new_header, final_df], ignore_index = True)
    
final_gs["acreage"] = [" ".join(x) if type(x) is list else x for x in final_gs["acreage"]]

for c in final_gs.columns:
    final_gs[c] = [r[:4999] if isinstance(r, str) and len(r) > 5000 else r for r in final_gs[c]]

# Summary of webpage links having errors
error_summary = pd.concat([new_header, final_df.loc[(final_df["error"] > 0)]], ignore_index = True)

error_summary["acreage"] = [" ".join(x) if type(x) is list else x for x in error_summary["acreage"]]

for c in error_summary.columns:
    error_summary[c] = [r[:4999] if isinstance(r, str) and len(r) > 5000 else r for r in error_summary[c]]

In [130]:
# Create a Google sheet to record the links with errors (Already created)
# if __name__ == '__main__':
#     # Pass: title
#     create("usace-test")

# I convert the lon/lat to string just for the storga in google sheet
# The format of lon/lat is still needed to be decided

# Update the scraped RSS feed and webpage info to the sheet
if __name__ == '__main__':
    # Pass: spreadsheet_id, range_name value_input_option and _values)
    update("1VYcmSCuvBMiRRpUusuulIcS4kjkXrTuFZLfKOV_IxkE",
            "new_orleans", 
           "USER_ENTERED",
           final_gs.values.tolist())

# Update the error messages to the sheet
if __name__ == '__main__':
    update("1VYcmSCuvBMiRRpUusuulIcS4kjkXrTuFZLfKOV_IxkE",
            "error", 
           "USER_ENTERED",
           error_summary.values.tolist())

11697 cells updated.
10888 cells updated.


## Achived

In [3]:
# Loop
error_collect = pd.DataFrame(columns = ["error_title", "error_link", "error_message"])

# %%time
for row in rss_feed.index:
    try:
        main_url = rss_feed.iloc[row, 1]
        req = requests.get(main_url)
        content = req.text # json structure
        soup = BeautifulSoup(content)
        
        # Get the pdf links
        
        def pdf_url():
            try:
                pdf_end = soup.findAll('a', {"class": "link"})[2]['href']
                pdf_url = "https://www.mvn.usace.army.mil" + pdf_end
            except Exception as pdf_error:
                print("ERROR when getting pdf links", pdf_error)
                
        # Check if the public notice is "Special Public Notice" or "Joint Public Notice"

        public_notice = soup.find_all("a", {"class":"link"})[2].get_text()

        if public_notice != "Public Notice and Drawings":
            expire_date = None
            applicant = None
            contractor = None
            location = None
            work_character = None
            mitigation = None
        else: 

            # Get expiration date

            expire_date = re.search(r'(?<=:\s).+', soup.find_all("div", "expire")[0].get_text()).group()

            # Extract webpage body: two types of structures

            body = soup.find_all("div", {"itemprop":"articleBody"})[0]
            if body.find("p") is None:
                body_text = body.get_text()  
            else:
                body_text = body.get_text().replace(u'\xa0', u'').replace("\n", "")

            applicant_contents = re.search(r'(?<=APPLICANT:)\s*.+(?=LOCATION)', body_text).group().strip()
            location = re.search(r'(?<=LOCATION OF WORK:)\s*.+(?=CHARACTER OF WORK)', body_text).group().strip()

            if "MITIGATION" in body_text:
                work_character = re.search(r'(?<=CHARACTER OF WORK:)\s*.+(?=MITIGATION)', body_text).group().strip()
                mitigation = re.search(r'(?<=MITIGATION:)\s*.+', body_text).group().strip()
            else:
                work_character = re.search(r'(?<=CHARACTER OF WORK:)\s*.+', body_text).group().strip()
                mitigation = None

            # Get applicant name and permitting process contractor

            if applicant_contents.find("c/o") != -1:
                applicant = re.search(r'.+?(?=\,* c/o)', applicant_contents).group().strip()
                # if any(w in applicant_contents for w in ["Attention", "Attn"]):
                contractor = re.search(r'(?<=c/o( |:)).+?(?=(, Post|, PO|, P\.O\.|, \d|,? [Aa][tT]))', applicant_contents).group().strip()
                # else:
                #     contractor = re.search(r'(?<=c/o( |:))\D+(?=(, |, P.O.))', applicant_contents).group()
            else:
                contractor = None
                # if any(w in applicant_contents for w in ["Attention", "Attn"]):
                applicant = re.search(r'.+?(?=(, Post|, PO|, P\.O\.|, \d|,? [Aa][tT]))', applicant_contents).group().strip()
                # else:
                #     applicant = re.search(r'\D+(?=, )', applicant_contents).group()

            # Get location
            
            
            #lon = re.findall(r'(?<=[-W])\s*\d{2}\.\d+', location)
            #lat = re.findall(r"(?<=[^-W][^-\d])\d{2}\.\d+", location)
            
        rss_feed.loc[row, empty_columns] = np.array([pdf_url, expire_date, applicant_contents, applicant, contractor, location, lon, lat, work_character, mitigation], dtype = "object")
    
    except Exception as error_message:
        error_row = [rss_feed.iloc[row, 0], rss_feed.iloc[row, 1], repr(error_message)]
        error_collect.loc[len(error_collect.index)] = error_row
        # print("ERROR: ", error_link, error)

rss_feed

,title,link,desc,date,pdf,expiration_date,applicant_full_info,applicant,contractor,location,lon,lat,work_character,mitigation
0,MVN-1999-02360-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,TIME EXTENSION FOR THE ENGLISH TURN DEVELOPMEN...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"English Turn Limited Partnership, Attention: J...",English Turn Limited Partnership,None,"At approximately Latitude: 29.90826, Longitude...",[89.952013],[29.90826],Continual operations to install and maintain f...,None
1,MVN-2020-00234-WPP,https://www.mvn.usace.army.mil/Missions/Regula...,MODIFICATION FOR MAINTENANCE DREDGING OF NAVIG...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"Port of Iberia, c/o: GIS Engineering, LLC, 450...",Port of Iberia,"GIS Engineering, LLC","Located in Iberia Parish, Section 34, Township...","[91.8412, 92.3127306]","[29.958239, 29.5109862]",A Department of Army permit was issued on Apri...,None
2,MVN-2021-00560-WQQ,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED CP2 LIQUEFIED NATURAL GAS FACILITY AN...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,"Venture Global CP2 LNG, LLC and Venture Global...","Venture Global CP2 LNG, LLC and Venture Global...",Environmental Resources Management,The proposed facility and pipeline are located...,[],"[85.4, 58.1]",To construct and operate (1) natural gas lique...,None
3,MVN-2023-00058-WII,https://www.mvn.usace.army.mil/Missions/Regula...,TEST WELLS FOR CARBON DIOXIDE STORAGE IN VERMI...,"Mon, 14 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,9/4/2023,ExxonMobil Low Carbon Solutions Onshore Storag...,ExxonMobil Low Carbon Solutions Onshore Storag...,Duplantis Design Group,Test Well #1 @ Lat. 29-46-57.31 N / Long. -92-...,"[57.31, 56.39, 44.82, 37.40]",[],Proposed geological investigations via install...,None
4,MVN-2008-01186-EPP,https://www.mvn.usace.army.mil/Missions/Regula...,AFTER-THE-FACT WEIGH STATION AND TRUCK LOADING...,"Mon, 07 Aug 2023 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,8/28/2023,"KV Enterprises, LLC, c/o Professional Engineer...","KV Enterprises, LLC",Professional Engineering & Environmental Consu...,"At approximately Latitude: 30.00584, Longitude...",[90.51464],[30.00584],"Clear, grade, excavate, and fill for a weigh s...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MVN-2020-00422-WS,https://www.mvn.usace.army.mil/Missions/Regula...,"BULKHEAD REPLACEMENT IN TERREBONNE PARISH, LOU...","Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/24/2020,"Savage Inland Marine, LLC, c/o: Mr. David Boud...","Savage Inland Marine, LLC",Mr. David Boudreaux,"Located in Terrebonne Parish, Section 14, Town...",[91.018125],[29.665289],The applicant proposes to install approximatel...,None
496,MVN-2019-00377-WI,https://www.mvn.usace.army.mil/Missions/Regula...,CONSTRUCT ONE BRINE PIPELINE AND ONE ETHYLENE ...,"Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/23/2020,"Boardwalk Louisiana Midstream, LLC, c/o Provid...","Boardwalk Louisiana Midstream, LLC",Providence Engineering and Environmental Group...,"In Iberville Parish, North 30.316461, West -91...",[91.304178],[30.316461],"Clear, grade, trench and temporarily backfill ...",None
497,MVN 2006-01716-CM,https://www.mvn.usace.army.mil/Missions/Regula...,PROPOSED CONSTRUCTION OF A RESIDNETIAL SUBDIVI...,"Mon, 04 May 2020 04:00:00 GMT",https://www.mvn.usace.army.mil/Portals/56/docs...,5/24/2020,"J. Breaux Enterprises, c/o Hydrik Wetlands Con...",J. Breaux Enterprises,Hydrik Wetlands Consultants,"Between West Colyell Creek and Beaver Branch, ...",[90.881303],[30.57454],The applicant has requested Department of the ...,None
498,MVN-2020-00393-CE,https://www.mvn.usace.army.mil/Missions/Regula...,MODIFICATION OF AN EXISTING BARGE FLEETING OPE...,"Mon, 